In [1]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [2]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'PcxOXWqBxWAx2beekgZ36KJfE'
        consumer_secret = 'c8i1VMTuN9H1zq5hXkATRk8Rk8E6QyJSJYC0NHkNXoO775Cgdl'
        access_token = '138549946-Grpm5kfVOko18ANo8PozWYxEohkWABVx7NnZxOHj'
        access_token_secret = 'PTcO7f1sUd9LuW2ioZlYkNHty6F8RNaaBaX2JszIXCGsJ'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                    |(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_subjetive(self, tweet):
        return TextBlob(self.clean_tweet(tweet)).sentiment.subjectivity

    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count, lang = 'es') 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
                parsed_tweet['subjetive'] = self.get_tweet_subjetive(tweet.text)
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))

In [3]:
#def main(): 
# creating object of TwitterClient Class 
api = TwitterClient() 
# calling function to get tweets 
tweets = api.get_tweets(query = 'Indulto', count = 2000) 

# picking positive tweets from tweets 
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
# percentage of positive tweets 
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
# picking negative tweets from tweets 
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
# percentage of negative tweets 
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
# percentage of neutral tweets 
print("Neutral tweets percentage: {} %"
    .format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets))) 

# printing first 5 positive tweets 
print("\n\nPositive tweets:") 
for tweet in ptweets[:10]: 
    print(tweet['text']) 

# printing first 5 negative tweets 
print("\n\nNegative tweets:") 
for tweet in ntweets[:10]: 
    print(tweet['text']) 

#if __name__ == "__main__": 
    # calling main function 
#    main() 

Positive tweets percentage: 2.6315789473684212 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 97.36842105263158 %


Positive tweets:
RT @IldegarGill: 110 indultos a 110 terroristas o pichones d eso demuestran el carácter humanista de Revolución Bolivariana. Una real dicta…
RT @indiodvenezuela: Ok, todo aquel de la oposición que acepta el indulto y sale libre hoy o mañana.... está reconociendo a .@NicolasMaduro…


Negative tweets:


In [8]:
for tweet in tweets:
    print(tweet['text'][tweet['text'].index(':')+1:])

 #31Ago #Indultos
@mavasquezg:"Siendo que el indulto es un perdón concedido a la persona que ha cometido un delito, no es pos…
 ¿ Si los de la oposición son tan dignos y firmes contra el Gobierno de Nicolas porque NO RECHAZAN el Indulto?
 MI ANÁLISIS PRIMERA PARTE: Las primeras declaraciones de los Diputados injustamente presos o perseguidos me hace entra…
 1) #AHORA En nuestra condición de Presidente de la Comisión para la Verdad, la Justicia, la Paz y la Tranquilidad Públ…
 El narcodictador @NicolasMaduro cometió el delito más grave contra la democracia al secuestrar y encarcelar cobarde e…
 Solo pasé por aquí a decir que si lo califican como indulto, siguen el juego al discurso de la administración de Maduro. 
Para…
 Yo los meto presos, yo los indulto. Aquí nadie tiene sentencia, por lo cual, creo, la figura sería la de sobreseimiento.…
//t.co/SEKSJL9RKz
 MADURO ES UN DESCARADO OTORGO INDULTO PRESIDENCIAL A PRESOS POLITICOS ( Es Justo) y CONDENA A LA CARCEL A PETROLEROS PRESOS…
 Asi

ValueError: substring not found